In [1]:
import numpy as np
from Session import Session
import pickle
import pandas as pd

In [2]:
def mrr( rank, label ):
    for i, item in enumerate(rank[:20]):
        if item == label:
            return 1.0/(i + 1)
    return 0

def recall( rank, label ):
    return int(label in rank[:20])
    
def pred2rank_result( pred ):
    top20_indices = np.argsort(pred)[-20:]
    top20_indices = np.flip(top20_indices, 0)
    rank_result = [ candidates[i] for i in top20_indices ]
    return rank_result


In [7]:
with open('../output/STAN_result.csv', 'r') as f:
    STAN_df = pd.read_csv(f)
    

In [11]:
df

,0,6409,146.212538
0,0,37169,30.326207
1,0,3644,25.904686
2,0,9878,18.231854
3,0,37146,16.778757
4,0,874,11.435755
...,...,...,...
1107409,55897,37169,0.338093
1107410,55897,37204,0.338093
1107411,55897,117,0.228783
1107412,55897,37172,0.223772
